In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#최종모델 구축

In [ ]:
# 1. input text -> 요약모델을 통한 한문장 생성
# 2. 요약된 한문장을 3개의 유사도 문장 추출
# 3. threshhold(유사도가 0.5이상)와 긍/부정 모델에 따른 최종 라벨 판단
# 4. input text 문장이 거짓 label로 판단된 경우 올바른 참 label에 맞는 문장 추출

### 필요한 패키지 설치

In [ ]:
# 패키지 설치
!pip install transformers
!pip install datasets
!pip install sentence_transformers

# 패키지 로드
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, BartModel,BartForConditionalGeneration
import math
import re
import logging
from datetime import datetime
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from nltk.translate.bleu_score import corpus_bleu
from transformers import pipeline, AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a13fe34c97ca5203d6b9b1ca8094a6010f5625174

In [ ]:
input = '문재인 아들 비리의혹에 관련한 기사~~'

###1.요약모델

In [ ]:
#기사 본문 정규식
def text_preprocess(text):
    text = re.sub("(<span class='quot[0-9]'>|\n\r\n|</span>|<br/>|<br />|([^0-9가-힣A-Za-z. ]))","",text)
    return text


# 요약모델 함수
def summarization(input):
  tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
  model_sm = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

  content = text_preprocess(input)
  text = re.sub(' +', ' ', content)
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  if len(input_ids) > 1024:
    summary_text = '해당 기사 본문 내용이 너무 지나치게 길어서 요약에 실패했습니다. 좀더 적은 양의 본문 내용을 뽑아주세요'
  else:
    summary_ids = model_sm.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
    summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  return summary_text

In [ ]:
sum_text = summarization(input)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
# 예시
sum_text = '문재인 아들이 비리 의혹이 있다.'
sum_text

'문재인 아들이 비리 의혹이 있다.'

###2. STS 모델 학습

In [ ]:
def train_sts():
  # GPU - A100

  # 모델 학습
  logging.basicConfig(

      format="%(asctime)s - %(message)s",
      datefmt="%Y-%m-%d %H:%M:%S",
      level=logging.INFO,
      handlers=[LoggingHandler()],
      )
  model_name = "klue/roberta-base"

  train_batch_size = 32
  num_epochs = 4
  model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

  embedding_model = models.Transformer(model_name)

  pooler = models.Pooling(
      embedding_model.get_word_embedding_dimension(),
      pooling_mode_mean_tokens=True,
      pooling_mode_cls_token=False,
      pooling_mode_max_tokens=False,
      )

  model_STS = SentenceTransformer(modules=[embedding_model, pooler])

  datasets = load_dataset("klue", "sts")
  testsets = load_dataset("kor_nlu", "sts")

  train_samples = []
  dev_samples = []
  test_samples = []

  for phase in ["train", "validation"]:
      examples = datasets[phase]

      for example in examples:
          score = float(example["labels"]["label"]) / 5.0

          inp_example = InputExample(
              texts=[example["sentence1"], example["sentence2"]],
              label=score,
              )

          if phase == "validation":
              dev_samples.append(inp_example)
          else:
              train_samples.append(inp_example)

  for example in testsets["test"]:
      score = float(example["score"]) / 5.0

      if example["sentence1"] and example["sentence2"]:
          inp_example = InputExample(
              texts=[example["sentence1"], example["sentence2"]],
              label=score,
              )

      test_samples.append(inp_example)

  train_dataloader = DataLoader(
      train_samples,
      shuffle=True,
      batch_size=train_batch_size,
      )
  train_loss = losses.CosineSimilarityLoss(model=model_STS)

  evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
      dev_samples,
      name="sts-dev",
      )

  warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
  logging.info(f"Warmup-steps: {warmup_steps}")

  model_STS.fit(
      train_objectives=[(train_dataloader, train_loss)],
      evaluator=evaluator,
      epochs=num_epochs,
      evaluation_steps=1000,
      warmup_steps=warmup_steps,
      output_path=model_save_path)

  #모델 저장
  torch.save(model_STS, f'/content/drive/MyDrive/conference/model.pt')

In [ ]:
train_sts()

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

###3. 요약문장 STS 모델 학습 후 유사도 문장 선정

In [ ]:
def test_sts(query, top_k):
  # 모델 설정
  model = torch.load('/content/drive/MyDrive/conference/model.pt')

  # 데이터 불러오기
  data = pd.read_csv('/content/drive/MyDrive/conference/final_data.csv', encoding='utf-8')
  # 비교 데이터 설정
  content = []
  for i in range(len(data)):

    content.append(str(data['contents'][i]))

  document_embeddings = model.encode(content)

  # 입력 데이터
  query_embedding = model.encode(query)

  # 입력 문장 - 문장 후보군 간 코사인 유사도 계산
  cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

  # 코사인 유사도 순으로 `top_k` 개 문장 추출
  top_results = torch.topk(cos_scores, k=top_k)

  num_top_results = sum(top_results[0] > 0.5)

  global cos_contents
  global label_contents
  cos_contents = []
  label_contents = []

  if num_top_results != 0:
    print(f"입력 문장: {query}")
    print(f"\n<입력 문장과 유사한 {num_top_results} 개의 문장>\n")

    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
      if score <= 0.5: break
      label = 'True' if data['label'][int(idx)] == 1 else 'False'
      cos_contents.append(content[idx])
      label_contents.append(label)
      print(f"{i+1}: {content[idx]} {'(유사도: {:.4f})'.format(score)} {'(진실 여부:{})'.format(label)}\n")

    cos_contents = cos_contents[0]
    label_contents = label_contents[0]

  elif num_top_results == 0:
    print(f"입력 문장: {query}")
    print(f"\n입력 문장과 유사한 문장이 {num_top_results} 개이므로 확인 불가합니다.\n")
#비슷한 주제 팩트

In [ ]:
def klue_nli(query, text_list):
  classifier = pipeline(
      "text-classification",
      model="Huffon/klue-roberta-base-nli",
      return_all_scores=True)
  tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

  global results
  results = classifier(f'{query} {tokenizer.sep_token} {text_list}')[0]
  return results

In [ ]:
def final_result(query, text, label_info, result):
  score_list = [ d['score'] for d in result]
  score_list

  if score_list.index(max(score_list)) == 1:
    score_list.sort(reverse=True)
    label_score = score_list[1]
    label = [item['label'] for item in result if item['score'] == label_score][0]

    if label == 'ENTAILMENT':
      print("label 결과가 {0}(수반)이므로 '{1}'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '{2}'과 동일하게 {3}로 결정된다.".format(label, query, text, label_info))
    elif label == 'CONTRADICTION':
      if label_info == 'True':
        label_info = 'False'
      else:
        label_info = 'True'
      print("label 결과가 {0}(모순)이므로 '{1}'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '{2}'와는 다르게 {3}로 결정된다.".format(label, query, text, label_info))

  else:
    score_list.sort(reverse=True)
    label_score = score_list[0]
    label = [item['label'] for item in result if item['score'] == label_score][0]
    if label == 'ENTAILMENT':
      print("label 결과가 {0}(수반)이므로 '{1}'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '{2}'과 동일하게 {3}로 결정된다.".format(label, query, text, label_info))
    else:
      if label_info == 'True':
        label_info = 'False'
      else:
        label_info = 'True'
      print("label 결과가 {0}(모순)이므로 '{1}'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '{2}'와는 다르게 {3}로 결정된다.".format(label, query, text, label_info))

  if (label == 'CONTRADICTION') & (label_info == 'False'):
    print("\n 따라서 올바른 문장은 '{0}'이다.".format(text))

In [ ]:
# query = sum_text, text = cos_contents, label = label_contents, result = results

In [ ]:
# sts return 값
test_sts(sum_text, 3)

입력 문장: 문재인 아들이 비리 의혹이 있다고 한다.

<입력 문장과 유사한 2 개의 문장>

1: 안철수 문재인 아들 5급 공무원에 특채는 사실 (유사도: 0.5470) (진실 여부:False)

2: 문재인 민주당 후보 아들이 고용정보원에서 휴직 중 승진했다 는 의혹  (유사도: 0.5393) (진실 여부:False)



In [ ]:
# sts 이후 사용해야할 내용
cos_contents, label_contents

('안철수 문재인 아들 5급 공무원에 특채는 사실', 'False')

In [ ]:
# nli return 값
klue_nli(sum_text, cos_contents)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'ENTAILMENT', 'score': 0.0005115066305734217},
 {'label': 'NEUTRAL', 'score': 0.9991310238838196},
 {'label': 'CONTRADICTION', 'score': 0.0003574293805286288}]

In [ ]:
# nli 이후 사용해야할 내용
results

[{'label': 'ENTAILMENT', 'score': 0.0005115066305734217},
 {'label': 'NEUTRAL', 'score': 0.9991310238838196},
 {'label': 'CONTRADICTION', 'score': 0.0003574293805286288}]

In [ ]:
final_result(sum_text, cos_contents, label_contents, results)

label 결과가 ENTAILMENT(수반)이므로 '문재인 아들이 비리 의혹이 있다고 한다.'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '안철수 문재인 아들 5급 공무원에 특채는 사실'과 동일하게 False로 결정된다.


#### 예시

In [ ]:
# 예시 1
sum_text = '중랑구에서는 서울장미축제를 개최하지 않는다'
test_sts(sum_text, 3)

입력 문장: 중랑구에서는 서울장미축제를 개최하지 않는다

<입력 문장과 유사한 1 개의 문장>

1: 중랑구는 꽃과 문화예술 먹거리가 어우러진 동네 혹은 지역 대표축제인 서울장미축제를 28일까지 개최한다. (유사도: 0.6744) (진실 여부:True)



In [ ]:
klue_nli(sum_text, cos_contents)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'ENTAILMENT', 'score': 0.00029482197714969516},
 {'label': 'NEUTRAL', 'score': 0.036649733781814575},
 {'label': 'CONTRADICTION', 'score': 0.9630554914474487}]

In [ ]:
final_result(sum_text, cos_contents, label_contents, results)

label 결과가 CONTRADICTION(모순)이므로 '중랑구에서는 서울장미축제를 개최하지 않는다'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '중랑구는 꽃과 문화예술 먹거리가 어우러진 동네 혹은 지역 대표축제인 서울장미축제를 28일까지 개최한다.'와는 다르게 False로 결정된다.

 따라서 올바른 문장은 '중랑구는 꽃과 문화예술 먹거리가 어우러진 동네 혹은 지역 대표축제인 서울장미축제를 28일까지 개최한다.'이다.


In [ ]:
# 예시 2 - 유사도 문장이 거짓이 나옴 -> 긍/부정에서는 contradiction(모순)이 더 높게 나옴 -> 따라서 실제 input은 거짓이 아니라 사실임.
sum_text = '문재인 아들이 비리 의혹이 없다.'
test_sts(sum_text, 3)
# 중랑구는 꽃과 문화예술 먹거리가 어우러진 동네 혹은 지역 대표축제인 서울장미축제를 28일까지 개최한다.
# 위의 실제 문장을 '중랑구는 서울장미축제를 28일까지 개최하지 않는다'

입력 문장: 문재인 아들이 비리 의혹이 없다.

<입력 문장과 유사한 1 개의 문장>

1: 안철수 문재인 아들 5급 공무원에 특채는 사실 (유사도: 0.5352) (진실 여부:False)



In [ ]:
klue_nli(sum_text, cos_contents)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'ENTAILMENT', 'score': 0.00033529079519212246},
 {'label': 'NEUTRAL', 'score': 0.9987083673477173},
 {'label': 'CONTRADICTION', 'score': 0.0009563304483890533}]

In [ ]:
final_result(sum_text, cos_contents, label_contents, results)

label 결과가 CONTRADICTION(모순)이므로 '문재인 아들이 비리 의혹이 없다.'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '안철수 문재인 아들 5급 공무원에 특채는 사실'와는 다르게 True로 결정된다.


In [ ]:
# 예시 3
sum_text = '문재인 아들이 비리 의혹이 있다고 한다.'
test_sts(sum_text, 3)

입력 문장: 문재인 아들이 비리 의혹이 있다고 한다.

<입력 문장과 유사한 2 개의 문장>

1: 안철수 문재인 아들 5급 공무원에 특채는 사실 (유사도: 0.5470) (진실 여부:False)

2: 문재인 민주당 후보 아들이 고용정보원에서 휴직 중 승진했다 는 의혹  (유사도: 0.5393) (진실 여부:False)



In [ ]:
klue_nli(sum_text, cos_contents)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'ENTAILMENT', 'score': 0.0005115066305734217},
 {'label': 'NEUTRAL', 'score': 0.9991310238838196},
 {'label': 'CONTRADICTION', 'score': 0.0003574293805286288}]

In [ ]:
final_result(sum_text, cos_contents, label_contents, results)

label 결과가 ENTAILMENT(수반)이므로 '문재인 아들이 비리 의혹이 있다고 한다.'문장에 대한 최종 결과는 유사도가 가장 높은 문장인 '안철수 문재인 아들 5급 공무원에 특채는 사실'과 동일하게 False로 결정된다.
